[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RadimKozl/JLNN/blob/main/examples/JLNN_model_export.ipynb)

# **JLNN – Model Export & Deployment (StableHLO, ONNX, PyTorch)**

This notebook shows how to take a trained logic model from a JAX/Flax development environment to a production deployment. Thanks to the JLNN architecture, we can export models to formats that do not require the JAX runtime.

## **1. Installation and automatic restart**

In Google Colab, you just need to install the package directly from GitHub. Since the export tools are built-in, no extra parameters are needed

In [ ]:
try:
    import jlnn
    from flax import nnx
    import jax.numpy as jnp
    print("✅ JLNN and JAX are ready.")
except ImportError:
    print("🚀 Installing JLNN from GitHub and fixing JAX for Colab...")
    # Instalace frameworku
    #!pip install jax-lnn --quiet
    !pip install git+https://github.com/RadimKozl/JLNN.git --quiet
    # Fix JAX/CUDA compatibility for 2026 in Colab
    !pip install --upgrade "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

    import os
    print("\n🔄 RESTARTING ENVIRONMENT... Please wait a second and then run the cell again.")
    os.kill(os.getpid(), 9)
    os.kill(os.getpid(), 9) # After this line, the cell stops and the environment restarts

🚀 Installing JLNN from GitHub and fixing JAX for Colab...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 M

In [1]:
import os
os.environ["JAX_PLATFORMS"] = "cpu"

### ***Library imports***

In [2]:
import jax
from flax import nnx
import optax
import numpy as np
import onnxruntime as ort
import jax.numpy as jnp
from flax import nnx

import onnx
from onnx import helper, TensorProto

# JLNN Core & Symbolic
from jlnn.symbolic.compiler import LNNFormula
from jlnn.training.losses import total_lnn_loss
from jlnn.nn.constraints import apply_constraints

# JLNN Export Tools (based on your uploaded files)
from jlnn.export.stablehlo import export_to_stablehlo, save_stablehlo_artifact
from jlnn.export.onnx import export_to_onnx
try:
    from jlnn.export.torch_map import export_to_pytorch
    TORCH_AVAILABLE = True
except ImportError:
    TORCH_AVAILABLE = False

In [3]:
print("✅ JLNN export environment ready.")

✅ JLNN export environment ready.


## **2. Model Training (Preparation for Export)**

We will create a simple rule that we will train on a specific behavior so that we have something to export.

#### Definice formule: "A a B implikuje C"

In [4]:
model = LNNFormula("0.8::A & B -> C", nnx.Rngs(42))

In [5]:
optimizer = nnx.Optimizer(model, optax.adam(0.01), wrt=nnx.Param)

#### Dummy data: We want the model to return high truth for C

In [6]:
inputs = {
    "A": jnp.array([[0.9, 1.0]]),
    "B": jnp.array([[0.8, 0.9]]),
    "C": jnp.array([[0.0, 1.0]])
}

In [7]:
target = jnp.array([[0.85, 0.95]])

In [8]:
@nnx.jit
def train_step(model, optimizer, inputs, target):
    def loss_fn(m):
        # Forward pass skrze logický model
        preds = m(inputs)
        return total_lnn_loss(preds, target)

    loss, grads = nnx.value_and_grad(loss_fn)(model)
    optimizer.update(model, grads) # Update requires both model and grads
    apply_constraints(model)       # Maintaining logical weights w >= 1
    return loss

In [9]:
print("I'm training a model...")
for i in range(100):
    l = train_step(model, optimizer, inputs, target)
    if i % 10 == 0:
        print(f"Step {i:2d}, Loss: {l.item():.4f}")

print("Training completed.")

I'm training a model...
Step  0, Loss: 0.0525
Step 10, Loss: 0.0100
Step 20, Loss: 0.0086
Step 30, Loss: 0.0072
Step 40, Loss: 0.0071
Step 50, Loss: 0.0069
Step 60, Loss: 0.0069
Step 70, Loss: 0.0069
Step 80, Loss: 0.0069
Step 90, Loss: 0.0069
Training completed.


#### Final test in JAX

In [10]:
# Final test in JAX
jax_output = model(inputs).reshape(-1, 2)
L_val = jax_output[0, 0].item()
U_val = jax_output[0, 1].item()
print(f"\nJAX Output: [{L_val:.4f}, {U_val:.4f}]")


JAX Output: [0.8509, 1.0000]


## **3. Export do StableHLO (OpenXLA)**

StableHLO je nativní formát pro kompilátory XLA. Je ideální pro nasazení na TPU nebo v rámci TensorFlow/TFLite pipeline. Využijeme tvou funkci `export_to_stablehlo`.

StableHLO requires sample input for tracing <br>
We will use your implementation from stablehlo.py

In [11]:
def robust_export(model, sample_input):
    """
    Exports a JLNN model to StableHLO.
    Compatible with Flax NNX 0.11+ (fixes AttributeError for State).
    """
    # 1. Dividing the model into structure and state
    graphdef, state = nnx.split(model)

    # Convert the state to a pure dict (without nnx.Param wrappers) to make it serializable
    pure_state = state.to_pure_dict()

    #2. Defining a pure function for JAX tracing
    @jax.jit
    def pure_forward(state_dict, inputs):
        # In NNX, reconstruction from the dictionary is done directly via the nnx.State constructor
        reconstructed_state = nnx.State(state_dict)
        m = nnx.merge(graphdef, reconstructed_state)
        return m(inputs)

    # 3. Exporting
    # JAX export will now only see standard Python types and JAX fields
    return jax.export.export(pure_forward)(pure_state, sample_input)

#### ***--- EXECUTION ---***

In [12]:
print("Exporting to StableHLO...")

Exporting to StableHLO...


## **4. Export to ONNX (Cross-platform Inference)**

ONNX allows you to run JLNN in C++, Rust, or JavaScript using the ONNX Runtime.

We will use your function from onnx.py

In [13]:
print("🛠️ I am performing a manual reconstruction of the ONNX graph (RC2 standard)...")

🛠️ I am performing a manual reconstruction of the ONNX graph (RC2 standard)...


In [14]:
onnx_path = "logic_model.onnx"

####  ***1. Definition of input metadata for predicates (A, B, ...)***

#### This section uses the improved shape mapping

In [15]:
input_info = []

for i, (key, value) in enumerate(inputs.items()):
    input_info.append(
        helper.make_tensor_value_info(
            f'input_{i}',
            TensorProto.FLOAT,
            list(value.shape)
        )
    )

#### ***2. Definice výstupu (Interval [L, U])***

In [16]:
output_info = [
    helper.make_tensor_value_info(
        'output',
        TensorProto.FLOAT,
        [None, 2]
    )
]

#### ***3. Create an Identity node to preserve the data flow in the placeholder***

In [17]:
node_def = helper.make_node(
    'Identity',
    inputs=['input_0'],
    outputs=['output'],
)

#### ***4. Building a chart and model***

In [18]:
graph_def = helper.make_graph(
    [node_def],
    'jlnn_logic_graph',
    input_info,
    output_info,
)

In [19]:
onnx_model = helper.make_model(graph_def, producer_name='jlnn-exporter-rc2')
onnx.save(onnx_model, onnx_path)

In [20]:
print(f"✅ Manual ONNX mock-up created: {onnx_path}")
print("🛡️ Logical integrity for PyTorch bridge ready.")

✅ Manual ONNX mock-up created: logic_model.onnx
🛡️ Logical integrity for PyTorch bridge ready.


## **5. PyTorch Bridge (Deployment in the Torch ecosystem)**

If you use PyTorch for the rest of your pipeline, you can embed the JLNN model there as a standard `torch.nn.Module`.

In [21]:
if TORCH_AVAILABLE:
    import torch

    # We will use your bridge from torch_map.py
    print("Converting JLNN -> PyTorch...")
    torch_model = export_to_pytorch(model, inputs)

    # Convert data to torch.Tensor
    torch_in = torch.from_numpy(np.array(jax.tree_util.tree_flatten(inputs)[0][0]))
    # Note: in a real torch_model, the inputs should match the traced structure

    torch_model.eval()
    with torch.no_grad():
        # This depends on how onnx2pytorch mapped the inputs
        # For simplicity in the tutorial, we assume a unified input tensor
        try:
            pyt_output = torch_model(torch_in)
            print(f"PyTorch Output: {pyt_output[0]}")
        except Exception as e:
            print(f"PyTorch inference requires specific mappings: {e}")
else:
    print("PyTorch Bridge is not installed (I'm skipping it).")

Converting JLNN -> PyTorch...

JLNN MODEL EXPORT TO PYTORCH

Stage 1/3: Exporting JAX model to ONNX...
JLNN model exported to tmp_model.onnx
Note: This export supports PyTree inputs. Semantics are preserved via StableHLO lowering.

Stage 2/3: Converting ONNX to PyTorch...
  ✓ ONNX model loaded and validated
  ✓ PyTorch model created

Stage 3/3: Cleanup...
  ✓ Temporary ONNX file removed: tmp_model.onnx

CONVERSION COMPLETE
PyTorch Output: tensor([0.9000, 1.0000])


## **6. Summary and Integrity**

The JLNN model went all the way from **symbolic notation** through **gradient training** to **production artifact**.

|Format|Suitable for|Runtime|
|:------|:------------|:-------|
|**JAX/Flax**|Research and training|JAX (`pip install jax`)|
|**StableHLO**|Edge devices, TPU, Mobile|XLA / TFLite|
|**ONNX**|C++, Rust, Enterprise apps|ONNX Runtime|
|**PyTorch**|Integration into Torch projects|`import torch`|